<a href="https://colab.research.google.com/github/paulaleite/aprendizagemDeMaquinaa2022/blob/main/Trabalho_Final_Aprendizagem_de_M%C3%A1quina.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho Final para Disciplina de Aprendizagem de Máquina
### Paula Torales Leite
Textinho explicando o problema

## Importações

In [1]:
# Processamento de Dados
import pandas as pd
import numpy as np
import scipy.stats
import seaborn as sns

# Modelagem e Aprendizagem de Máquina
from sklearn.metrics.pairwise import cosine_similarity
import operator
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix

## Preparação dos Dados
União dos dados da base de avaliações e de desafios, para que seja possível construir a base que será utilizada durante o processo.
Remoção dos dados duplicados, bem como a escolha do Challenge que será focado.

In [2]:
# Leitura de dadas
avaliacoes = pd.read_csv('./dados/avaliacoes.csv', sep=';')
avaliacoes.head()

,userID,challengeID,objectiveID,rating
0,24,4,30,2
1,8,6,25,3
2,7,3,31,2
3,30,4,5,4
4,27,7,49,1


In [3]:
# Informação sobre os dados
avaliacoes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 833 entries, 0 to 832
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype
---  ------       --------------  -----
 0   userID       833 non-null    int64
 1   challengeID  833 non-null    int64
 2   objectiveID  833 non-null    int64
 3   rating       833 non-null    int64
dtypes: int64(4)
memory usage: 26.2 KB


In [4]:
print('O dataset de avaliações tem', avaliacoes['userID'].nunique(), 'usuários únicos')
print('O dataset de avaliações tem', avaliacoes['challengeID'].nunique(), 'desafios únicos')
print('O dataset de avaliações tem', avaliacoes['objectiveID'].nunique(), 'objetivos únicos')
print('O dataset de avaliações tem', avaliacoes['rating'].nunique(), 'unique ratings')
print('As avalições únicas são', sorted(avaliacoes['rating'].unique()))

O dataset de avaliações tem 30 usuários únicos
O dataset de avaliações tem 8 desafios únicos
O dataset de avaliações tem 55 objetivos únicos
O dataset de avaliações tem 5 unique ratings
As avalições únicas são [1, 2, 3, 4, 5]


In [5]:
objetivos = pd.read_csv('./dados/objetivos.csv', sep=';')
objetivos.head()

,objectiveID,title
0,1,Objetivo1
1,2,Objetivo2
2,3,Objetivo3
3,4,Objetivo4
4,5,Objetivo5


In [6]:
# Merge das avaliações e objetivos
df = pd.merge(avaliacoes, objetivos, on='objectiveID', how='inner')
df.head()

,userID,challengeID,objectiveID,rating,title
0,24,4,30,2,Objetivo30
1,26,4,30,5,Objetivo30
2,12,2,30,2,Objetivo30
3,17,6,30,3,Objetivo30
4,12,2,30,1,Objetivo30


In [7]:
df_duplicados = df.copy()

In [8]:
df_duplicados.duplicated().sum()

5

In [9]:
df_duplicados = df_duplicados.drop_duplicates()

In [10]:
df_duplicados.duplicated().sum()

0

In [11]:
desafio_atual = 3

df_filtrado = df_duplicados[df_duplicados.challengeID == desafio_atual]
df_filtrado.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114 entries, 12 to 829
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   userID       114 non-null    int64 
 1   challengeID  114 non-null    int64 
 2   objectiveID  114 non-null    int64 
 3   rating       114 non-null    int64 
 4   title        114 non-null    object
dtypes: int64(4), object(1)
memory usage: 5.3+ KB


## Análise Exploratória
Criação da Matriz advinda do dataset de base.
Normalização dos dados da matriz.
Construção da similiaridade por item utilizando a correlação de Pearson

In [12]:
matriz_id = df_filtrado.pivot_table(index='objectiveID', columns='userID', values='rating')
matriz_id.head()

userID,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
objectiveID,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,3.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN
5,3.0,NaN,NaN,2.5,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
matriz_id_norm = matriz_id.subtract(matriz_id.mean(axis=1), axis = 0)
matriz_id_norm.head()

userID,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
objectiveID,,,,,,,,,,,,,,,,,,,,,
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.750,NaN,NaN,...,NaN,1.25,NaN,0.25,NaN,NaN,NaN,NaN,NaN,NaN
5,0.625,NaN,NaN,0.125,NaN,NaN,NaN,-1.375,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,-1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,-1.00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
matriz = df_filtrado.pivot_table(index='title', columns='userID', values='rating')
matriz.head()

userID,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
title,,,,,,,,,,,,,,,,,,,,,
Objetivo10,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo11,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,5.0,...,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN
Objetivo12,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
Objetivo13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo15,NaN,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
matriz_norm = matriz.subtract(matriz.mean(axis=1), axis = 0)
matriz_norm.head()

userID,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
title,,,,,,,,,,,,,,,,,,,,,
Objetivo10,-2.333333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.666667,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo11,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
Objetivo12,NaN,NaN,NaN,NaN,NaN,NaN,0.500000,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.5
Objetivo13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo15,NaN,NaN,NaN,NaN,NaN,NaN,1.333333,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
similiaridade_de_item = matriz_norm.T.corr()
similiaridade_de_item.head()

title,Objetivo10,Objetivo11,Objetivo12,Objetivo13,Objetivo15,Objetivo16,Objetivo17,Objetivo19,Objetivo2,Objetivo21,...,Objetivo50,Objetivo51,Objetivo52,Objetivo53,Objetivo54,Objetivo55,Objetivo6,Objetivo7,Objetivo8,Objetivo9
title,,,,,,,,,,,,,,,,,,,,,
Objetivo10,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo11,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo12,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Objetivo15,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Classificação
Construção da função de Recomendação de Objetivos

In [17]:
def criando_matriz_csr():
  M = len(df['objectiveID'].unique())
  N = len(df['userID'].unique())

  mapeamento_usuario = dict(zip(np.unique(df["userID"]), list(range(N))))
  mapeamento_objetivo = dict(zip(np.unique(df["objectiveID"]), list(range(M))))

  mapeamento_inv_usuario = dict(zip(list(range(N)), np.unique(df["userID"])))
  mapeamento_inv_objetivo = dict(zip(list(range(M)), np.unique(df["objectiveID"])))

  indice_usuario = [mapeamento_usuario[i] for i in df['userID']]
  indice_objetivo = [mapeamento_objetivo[i] for i in df['objectiveID']]

  X = csr_matrix((df['rating'], (indice_objetivo, indice_usuario)), shape=(M, N))

  return X, mapeamento_usuario, mapeamento_objetivo, mapeamento_inv_usuario, mapeamento_inv_objetivo

In [18]:
def busca_objetivos_similares(objectivoID, k, metric='cosine', show_distance=False):
    X, mapeamento_usuario, mapeamento_objetivo, mapeamento_inv_usuario, mapeamento_inv_objetivo = criando_matriz_csr()

    ids_vizinhos = []
    
    ind_objetivo = mapeamento_objetivo[objectivoID]
    vec_objetivo = X[ind_objetivo]
    k += 1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    vec_objetivo = vec_objetivo.reshape(1,-1)
    neighbour = kNN.kneighbors(vec_objetivo, return_distance=show_distance)
    for i in range(0,k):
        n = neighbour.item(i)
        ids_vizinhos.append(mapeamento_inv_objetivo[n])
    ids_vizinhos.pop(0)
    return ids_vizinhos

In [19]:
def recomeadacao_baseada_knn(userID_escolhido, objetivoID, k):
  # IDs dos objetivos que o usuário não avaliou
  id_objetivo_sem_avaliacao = pd.DataFrame(matriz_id_norm[userID_escolhido].isna()).reset_index()
  id_objetivo_sem_avaliacao = id_objetivo_sem_avaliacao[id_objetivo_sem_avaliacao[userID_escolhido]==True]['objectiveID'].values.tolist()

  # Dicionário para salvar os objetivos não avaliados e o resultado da predição
  predicao_objetivos = []

  for objetivo_atual in id_objetivo_sem_avaliacao:
    objetivos_encontrados = busca_objetivos_similares(objetivoID, k=10)
    predicao_objetivos = objetivos_encontrados

  return predicao_objetivos

In [20]:
def rec_baseada_item(userID_escolhido, numero_itens_similares, numero_recomendacoes, metric='cosine', show_distance=False):
  # Objetivos que o usuário não avaliou
  objetivos_sem_avaliacao = pd.DataFrame(matriz_norm[userID_escolhido].isna()).reset_index()
  objetivos_sem_avaliacao = objetivos_sem_avaliacao[objetivos_sem_avaliacao[userID_escolhido]==True]['title'].values.tolist()

  # Objetivos que o usuário avaliou
  objetivos_avaliados = pd.DataFrame(matriz_norm[userID_escolhido].dropna(axis=0, how='all')\
                            .sort_values(ascending=False))\
                            .reset_index()\
                            .rename(columns={userID_escolhido:'rating'})

  # Dicionário para salvar os objetivos não avaliados e o resultado da predição
  avaliacao_predicao = {}
         
  for objetivo_atual in objetivos_sem_avaliacao: 
    # Cálcula da similaridade do objetivo atual com outros filmes
    # Tenho que verificar se o objetivo_atual realmente existe dentro da matriz similiaridade_de_item
    similaridade_objetivo_atual = similiaridade_de_item[[objetivo_atual]].reset_index().rename(columns={objetivo_atual:'score_similaridade'})

    # Rank das similaridades entre os objetivos do usuário selecionado e o objetivo atual
    objetivos_avaliados_similaridade = pd.merge(left=objetivos_avaliados, 
                                                right=similaridade_objetivo_atual, 
                                                on='title', 
                                                how='inner')\
                                        .sort_values('score_similaridade', ascending=False)[:numero_itens_similares]

    # Cálculo da classificação prevista usando a média ponderada das pontuações de similaridade e as classificações do usuário 1
    predicao_avaliacao = round(np.average(objetivos_avaliados_similaridade['rating'], 
                                        weights=objetivos_avaliados_similaridade['score_similaridade']), 6)
    
    # Salvando a avaliação prevista no dicionário, apenas no caso em que não exista NaN
    if pd.isna(predicao_avaliacao) == False:
      avaliacao_predicao[objetivo_atual] = predicao_avaliacao

  # Retorno das avaliações principais
  return sorted(avaliacao_predicao.keys(), key=operator.itemgetter(1), reverse=True)[:numero_recomendacoes]

In [21]:
objetivoID = 24
usuarioID = 6

In [45]:
titulos_objetivos = dict(zip(df_filtrado['objectiveID'], df_filtrado['title']))
titulos_objetivos

objetivos_recomendados_knn = recomeadacao_baseada_knn(userID_escolhido = usuarioID, objetivoID = objetivoID, k = 10)
objetivos_recomendados_knn

titulo_objetivos_selecionados = []

for i in objetivos_recomendados_knn:
  if i in titulos_objetivos:
    titulo_objetivos_selecionados.append(titulos_objetivos[i])

print(titulo_objetivos_selecionados)

['Objetivo9', 'Objetivo51', 'Objetivo38', 'Objetivo7', 'Objetivo5', 'Objetivo45', 'Objetivo10', 'Objetivo15']


In [46]:
objetivos_recomendados = rec_baseada_item(userID_escolhido = usuarioID, numero_itens_similares = 1, numero_recomendacoes = 5)
objetivos_recomendados

['Objetivo37', 'Objetivo46']

In [47]:
if len(titulo_objetivos_selecionados) >= len(objetivos_recomendados):
    print(titulo_objetivos_selecionados)
else:
    print(objetivos_recomendados)

['Objetivo9', 'Objetivo51', 'Objetivo38', 'Objetivo7', 'Objetivo5', 'Objetivo45', 'Objetivo10', 'Objetivo15']
